In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
import pandas as pd
import numpy as np
import ast
import utils
from sqlalchemy import create_engine

In [11]:
%%time 
df_review = pd.read_json(
    "yelp_dataset/yelp_academic_dataset_review.json", lines=True
)

CPU times: user 46.9 s, sys: 2min 53s, total: 3min 40s
Wall time: 8min 41s


In [13]:
df_review

(8635403, 9)

In [16]:
%%time
df_review = utils.insert_text_length_column(df_review)

writing text length column
CPU times: user 10.4 s, sys: 8.49 s, total: 18.9 s
Wall time: 22.9 s


In [18]:
df_review.to_sql("reviews", con=engine, if_exists="replace")

In [3]:
from constants import STANFORD, CAMBRIDGE, VANCOUVER

In [19]:
utils.make_index('yelp.db', 'tips', 'business_id')
utils.make_index('yelp.db', 'reviews', 'business_id')
utils.make_index('yelp.db', 'businesses', 'business_id')

In [21]:
%%time
engine = create_engine("sqlite:///yelp.db", echo=False)
query = f"""WITH close_businesses AS (SELECT 
                business_id,
                name,
                address,
                city,
                state,
                stars,
                review_count,
                distance_to_cambridge
            FROM businesses
            WHERE distance_to_cambridge < 10
            AND categories LIKE "%Bubble Tea%"
            )
            SELECT
                close_businesses.business_id,
                name,
                address,
                city,
                state,
                close_businesses.stars as overall_star,
                review_count,
                distance_to_cambridge,
                reviews.stars,
                useful,
                funny,
                cool,
                text,
                date
            FROM close_businesses
            LEFT JOIN reviews 
                ON close_businesses.business_id = reviews.business_id
            """
df_close = pd.read_sql(
    query,
    con=engine,
)
df_close.to_csv("data/boston_boba_reviews.csv", index = False)

CPU times: user 235 ms, sys: 217 ms, total: 452 ms
Wall time: 1.99 s


# adhoc scripts

In [140]:
%%time
engine = create_engine("sqlite:///yelp.db", echo=False)
query = f"""WITH sentiment AS (SELECT 
                business_id, 
                AVG(polarity > 0) as positive, 
                AVG(polarity < 0) as negative, 
                AVG(polarity = 0) as neutral, 
                COUNT(business_id) as count,
                MIN(text) as first_text
            FROM tips
            WHERE text_length > 0
            GROUP BY business_id
            )
            SELECT 
                businesses.business_id,
                name, 
                address, 
                positive, 
                negative, 
                neutral, 
                count, 
                first_text,
                review_count, 
                categories, 
                distance_to_cambridge
            FROM sentiment
            LEFT JOIN businesses 
                ON businesses.business_id = sentiment.business_id
            WHERE distance_to_cambridge < 10
            AND categories LIKE "%Bubble Tea%"
            AND COUNT > 5
            ORDER BY positive DESC
            """
df = pd.read_sql(
    query,
    con=engine,
)
df

CPU times: user 1.11 s, sys: 453 ms, total: 1.56 s
Wall time: 1.57 s


,business_id,name,address,positive,negative,neutral,count,first_text,review_count,categories,distance_to_cambridge
0,cCHCD5LutCS5QRTNQlMDXg,Bubble Nation,456A Massachusetts Ave,1.000000,0.000000,0.000000,6,Amazing place I got a wrong order but they mad...,53,"Coffee & Tea, Food, Desserts, Bubble Tea",3.559409
1,jK4b6W3ApA2w9-WH0WrPrg,Kung Fu Tea,246 Moody St,1.000000,0.000000,0.000000,7,Easy to order-the green tea with milk was so g...,43,"Juice Bars & Smoothies, Bubble Tea, Food, Coff...",6.487826
2,kXCBLqxFHmYstXO3M6O15Q,Wok n Talk,525 Washington St,1.000000,0.000000,0.000000,7,A healthier spot to get your favorite Chinese ...,55,"Restaurants, Food, Bubble Tea, Chinese, Asian ...",3.199899
3,QtT93CbzAfzBxPpIl_6J4Q,Cha Shu Coffee and Bubble Tea,62 Summer St,1.000000,0.000000,0.000000,19,..... GOAT! ( greatest of all time ) facts,89,"Juice Bars & Smoothies, Sandwiches, Bubble Tea...",4.228828
4,Eg2wMDI7tttRZQ6u2usKNw,Coco Leaf on Newbury,303 Newbury St,0.916667,0.000000,0.083333,12,5% discount when you pay with cash!!!,89,"Restaurants, Juice Bars & Smoothies, Food, Bub...",2.103792
...,...,...,...,...,...,...,...,...,...,...,...
64,mlxQmZli1YGIArgRSLsICQ,Chatime,31 Harrison Ave,0.433333,0.200000,0.366667,30,20 minutes to make tea--- hmmm,236,"Bubble Tea, Coffee & Tea, Food",2.899948
65,EA4k17VsQlVNd893Xd8UWQ,Reign Drink Lab,1370 Dorchester Ave,0.400000,0.400000,0.200000,10,Amazing nitro cold brew!!,104,"Bubble Tea, Juice Bars & Smoothies, Coffee & T...",5.456567
66,64rFVPsZsslTvmvg1KlQuA,Vivi Bubble Tea Cafe Quincy,389 Hancock St,0.375000,0.281250,0.343750,32,"10/10 they have amazing chicken, and is worth ...",196,"Food, Juice Bars & Smoothies, Internet Cafes, ...",8.114856
67,g2H58GCWZlYq6BcOywDi7g,Poké Bento,827 Boylston St,0.357143,0.357143,0.285714,14,5-2-18 White rice was undercooked. Too bad I d...,114,"Food, Hawaiian, Desserts, Poke, Restaurants, B...",2.193259


In [141]:
%%time
engine = create_engine("sqlite:///yelp.db", echo=False)
query = f"""SELECT *
            FROM tips
            WHERE business_id = "QtT93CbzAfzBxPpIl_6J4Q"
        """
df = pd.read_sql(
    query,
    con=engine,
)
df

CPU times: user 117 ms, sys: 55.2 ms, total: 173 ms
Wall time: 172 ms


,index,user_id,business_id,text,date,compliment_count,text_length,polarity,subjectivity
0,116732,zIb8Pr0e4h0jlWFqEfr7tw,QtT93CbzAfzBxPpIl_6J4Q,Why are they closing? They are the best boba t...,2018-01-28 17:56:07.000000,0,12,1.000000,0.300000
1,240332,j4EE50lJyjlZ9RgARLO1Og,QtT93CbzAfzBxPpIl_6J4Q,Ask for what's good to try out. They have good...,2014-12-31 23:49:35.000000,0,10,0.700000,0.600000
2,288961,I4c1bMGS2ZtkJShR8FZaTw,QtT93CbzAfzBxPpIl_6J4Q,Great quick breakfast stop in Malden.,2017-08-12 13:59:27.000000,0,5,0.566667,0.625000
3,450951,j4EE50lJyjlZ9RgARLO1Og,QtT93CbzAfzBxPpIl_6J4Q,Service is Good and the quality of tea is grea...,2014-12-17 01:26:25.000000,0,12,0.600000,0.483333
4,474249,N61hkQVfKF0GXP3UIVnbWw,QtT93CbzAfzBxPpIl_6J4Q,Grab a loyalty card- 6th drink is free,2017-01-24 13:14:48.000000,0,7,0.400000,0.800000
5,544177,8LquCKKe0w-t5q89wVvHPw,QtT93CbzAfzBxPpIl_6J4Q,Best place for boba,2016-07-30 21:17:16.000000,0,3,1.000000,0.300000
6,618390,n9igsmtzYYC0vl8klOLDZw,QtT93CbzAfzBxPpIl_6J4Q,Ryan's the best! Always feel at home there. Re...,2016-10-15 02:01:35.000000,0,10,0.625000,0.300000
7,695489,MCuos7MaHvKKuDiCQtybcw,QtT93CbzAfzBxPpIl_6J4Q,"Great coffee! Bold and strong, without the bi...",2014-12-15 13:51:56.000000,0,8,0.588889,0.716667
8,699485,lZRLtrlOBaye-O_GgaaBtQ,QtT93CbzAfzBxPpIl_6J4Q,"Great drinks, food, and service! A neighborhoo...",2017-08-25 11:38:28.000000,0,7,0.750000,0.875000
9,738801,2IMqj2mDZjuLiVIu6Ivzgw,QtT93CbzAfzBxPpIl_6J4Q,The owner Ryan is very nice and welcoming! Mat...,2017-04-01 16:40:08.000000,0,13,0.987500,1.000000


In [146]:
%%time
engine = create_engine("sqlite:///yelp.db", echo=False)
query = f"""WITH sentiment AS (SELECT 
                business_id, 
                COUNT(business_id) as count,
                MIN(text) as first_text
            FROM reviews
            GROUP BY business_id
            )
            SELECT 
                businesses.business_id,
                name, 
                address, 
                first_text,
                review_count, 
                categories, 
                distance_to_cambridge
            FROM sentiment
            LEFT JOIN businesses 
                ON businesses.business_id = sentiment.business_id
            WHERE distance_to_cambridge < 10
            AND categories LIKE "%Bubble Tea%"
            AND COUNT > 5
            """
df = pd.read_sql(
    query,
    con=engine,
)
df

CPU times: user 12.4 s, sys: 19.7 s, total: 32 s
Wall time: 59.7 s


,business_id,name,address,first_text,review_count,categories,distance_to_cambridge
0,8xNY-EwTl6j1Wp4XcBZg8Q,Dumpling Daughter,37 Center St,3 Star seems low it's really 3 1/2 but that's ...,339,"Chinese, Bubble Tea, Food, Restaurants",9.788131
1,9Ih-4eMu4qaOL0E05JrOwA,Kung Fu Tea,1916 Beacon St,Always way too empty for the service and quali...,97,"Juice Bars & Smoothies, Food, Coffee & Tea, Bu...",3.254382
2,N6euK1sd4ktlpp1nFdORAQ,Hearty Teriyaki Newton,67 Crafts St,A VERY welcome addition to Newton. Fairly basi...,77,"Japanese, Bubble Tea, Asian Fusion, Restaurant...",4.877459
3,MqvMaY-0nmmZYf0B7tDBbQ,Boba Me,1520 Tremont St,A new boba/tea spot in the Mission Hill area. ...,50,"Vietnamese, Restaurants, Juice Bars & Smoothie...",2.898445
4,P6u9VBwU20tkfEKIlmrOTA,Patty Chen's Dumpling Room,907 Main St,"........sigh. I don't know why I came here, ev...",200,"Specialty Schools, Education, Ethnic Food, Caf...",0.863307
...,...,...,...,...,...,...,...
112,v972-kNc67HyGqdpqoCHSQ,Gong Cha,154 Harvard Ave,"A staple for bubble tea in Boston, Gongcha is ...",105,"Bubble Tea, Food, Taiwanese, Restaurants",1.879708
113,18aDgJE0XsOQ474ZddwUzw,Sweet Bubble Truck,,Found the Sweet Bubble Truck in the SOWA Marke...,14,"Food Trucks, Food, Bubble Tea",2.212033
114,U2NMp63yiFaFPcy6URHLiw,Chatime Boston 2nd Store,18 Hudson St,Chatime has consistently good bubble tea. This...,28,"Food, Coffee & Tea, Bubble Tea",2.998857
115,7LFSgs2v3Kzjqd8SaRrB5A,Banh Mi House,48 Winter St,3.5-4 stars. \n\nBanh Mi House does a good job...,114,"Vietnamese, Food, Bubble Tea, Restaurants, San...",2.731719


In [150]:
%%time
engine = create_engine("sqlite:///yelp.db", echo=False)
query = f"""WITH close_businesses AS (SELECT 
                business_id,
                name,
                address,
                city,
                state,
                stars,
                review_count,
                distance_to_cambridge
            FROM businesses
            WHERE distance_to_cambridge < 10
            AND categories LIKE "%Bubble Tea%"
            )
            SELECT *
            FROM close_businesses
            LEFT JOIN reviews 
                ON close_businesses.business_id = reviews.business_id
            """
df_close = pd.read_sql(
    query,
    con=engine,
)
df_close

CPU times: user 1min 28s, sys: 3min 5s, total: 4min 33s
Wall time: 4min 34s


,business_id,name,address,city,state,stars,review_count,distance_to_cambridge,index,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,8xNY-EwTl6j1Wp4XcBZg8Q,Dumpling Daughter,37 Center St,Weston,MA,4.0,339,9.788131,3361,bL94jxvGWrrNbOhR-B1FXw,KEaZslOcBoPsqdhmrcIvTA,8xNY-EwTl6j1Wp4XcBZg8Q,5,0,0,0,"My family ordered scallion pancakes, soup dump...",2016-01-17 17:30:01.000000
1,8xNY-EwTl6j1Wp4XcBZg8Q,Dumpling Daughter,37 Center St,Weston,MA,4.0,339,9.788131,5619,Tkb7-DMQW3OscuDcR9G-VQ,1dbJJm-ayhZj-CDpkOrqkQ,8xNY-EwTl6j1Wp4XcBZg8Q,5,0,0,0,"I went to Taiwan two years ago for work, for 9...",2016-04-23 02:26:34.000000
2,8xNY-EwTl6j1Wp4XcBZg8Q,Dumpling Daughter,37 Center St,Weston,MA,4.0,339,9.788131,6882,6he-hRb4V1u4JKYT4iLRdA,Zd2MiIfCHT52WXoUKtYONg,8xNY-EwTl6j1Wp4XcBZg8Q,5,1,0,1,I could not recommend this place more- the wom...,2018-03-04 01:14:07.000000
3,8xNY-EwTl6j1Wp4XcBZg8Q,Dumpling Daughter,37 Center St,Weston,MA,4.0,339,9.788131,7554,ViGAo5wPFjT2AzflTILSdA,m6PLbFZlhMf6kWiNsmMdog,8xNY-EwTl6j1Wp4XcBZg8Q,5,0,0,0,Simply delicious! My husband and I love this ...,2015-04-28 20:29:44.000000
4,8xNY-EwTl6j1Wp4XcBZg8Q,Dumpling Daughter,37 Center St,Weston,MA,4.0,339,9.788131,11137,IILGsRvFPEOTxnviz7VXyg,8qWb_j_2qk7kWtM7tWuA0w,8xNY-EwTl6j1Wp4XcBZg8Q,5,0,0,0,"I don't give many 5 stars, but the food here d...",2016-01-17 20:43:16.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14103,W_YBG1lPLB-cehQbwtXKow,Chatime,333 Huntington Ave,Boston,MA,2.5,74,2.514335,8523131,JhEk5PlwO54ZLgvcvl5jxA,xd6hVDLy9JzKqLhuGZjkTQ,W_YBG1lPLB-cehQbwtXKow,1,0,1,0,This location has one of the worst supply mana...,2017-03-28 22:34:52.000000
14104,W_YBG1lPLB-cehQbwtXKow,Chatime,333 Huntington Ave,Boston,MA,2.5,74,2.514335,8526416,fo2_PGNP5YNDQEKgX1wa6A,G9MotztINI1hcKpKvljGBA,W_YBG1lPLB-cehQbwtXKow,1,0,0,0,Meh. Nothing to write home about. Ordered a Th...,2019-07-22 22:23:16.000000
14105,W_YBG1lPLB-cehQbwtXKow,Chatime,333 Huntington Ave,Boston,MA,2.5,74,2.514335,8549684,qr0Gk6CpZj36KoXKBNlOcA,jz4gyRur6Yq4VGlINthdmg,W_YBG1lPLB-cehQbwtXKow,5,0,0,0,This is probably my favorite tea place and bub...,2019-06-02 17:30:37.000000
14106,W_YBG1lPLB-cehQbwtXKow,Chatime,333 Huntington Ave,Boston,MA,2.5,74,2.514335,8583857,Ls_9OWX_ZzgFWcn41Aq3Cg,DV8qKxGam_MsHt2EdsSk8w,W_YBG1lPLB-cehQbwtXKow,3,0,0,0,Mainly a tea place. Most of the add-ons are hi...,2019-05-28 00:35:05.000000


In [158]:
%%time
engine = create_engine("sqlite:///yelp.db", echo=False)
query = f"""WITH gc AS (SELECT *
            FROM tips
            WHERE business_id = "v972-kNc67HyGqdpqoCHSQ")
            SELECT *
            FROM gc
            LEFT JOIN businesses
            ON businesses.business_id = gc.business_id
            """
tips_gc = pd.read_sql(
    query,
    con=engine,
)
tips_gc

CPU times: user 1.27 s, sys: 2.21 s, total: 3.48 s
Wall time: 5.32 s


,index,user_id,business_id,text,date,compliment_count,text_length,polarity,subjectivity,index,...,longitude,stars,review_count,is_open,attributes,categories,hours,distance_to_cambridge,distance_to_stanford,distance_to_vancouver
0,438276,c075SXdoIxYK01i5R9ev3Q,v972-kNc67HyGqdpqoCHSQ,Subscribe to Gong Cha's loyalty program by giv...,2018-08-02 02:40:10.000000,0,22,0.0,0.0,152333,...,-71.131533,3.5,105,1,"{'RestaurantsPriceRange2': '2', 'BusinessParki...","Bubble Tea, Food, Taiwanese, Restaurants","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",1.879708,2692.798799,2504.488825
1,517876,0h1chtgnLpNpVy4shz3FWw,v972-kNc67HyGqdpqoCHSQ,Get the strawberry flavored milk foam! It's wo...,2019-09-24 00:50:40.000000,0,7,0.0,0.0,152333,...,-71.131533,3.5,105,1,"{'RestaurantsPriceRange2': '2', 'BusinessParki...","Bubble Tea, Food, Taiwanese, Restaurants","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",1.879708,2692.798799,2504.488825
2,890271,IcgbOt1-S3M6HfdQzY3mzA,v972-kNc67HyGqdpqoCHSQ,Bubble tea was good but you have to pay separa...,2018-07-29 18:18:10.000000,0,12,0.7,0.6,152333,...,-71.131533,3.5,105,1,"{'RestaurantsPriceRange2': '2', 'BusinessParki...","Bubble Tea, Food, Taiwanese, Restaurants","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",1.879708,2692.798799,2504.488825
3,1146600,otxlfcAiA5RmTgKQNM1_tA,v972-kNc67HyGqdpqoCHSQ,Good,2019-10-05 21:08:10.000000,0,0,0.7,0.6,152333,...,-71.131533,3.5,105,1,"{'RestaurantsPriceRange2': '2', 'BusinessParki...","Bubble Tea, Food, Taiwanese, Restaurants","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",1.879708,2692.798799,2504.488825
4,1149119,KHnn8Mww3sqlLuq_lwoPaQ,v972-kNc67HyGqdpqoCHSQ,yeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeet,2019-12-25 22:38:06.000000,0,0,0.0,0.0,152333,...,-71.131533,3.5,105,1,"{'RestaurantsPriceRange2': '2', 'BusinessParki...","Bubble Tea, Food, Taiwanese, Restaurants","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",1.879708,2692.798799,2504.488825
5,1161558,u_hS-gSZgf3jvDDMKFlteA,v972-kNc67HyGqdpqoCHSQ,Always a good experience and that's why I'm a ...,2018-08-11 19:52:59.000000,0,10,0.7,0.6,152333,...,-71.131533,3.5,105,1,"{'RestaurantsPriceRange2': '2', 'BusinessParki...","Bubble Tea, Food, Taiwanese, Restaurants","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",1.879708,2692.798799,2504.488825


In [5]:
%%time
engine = create_engine("sqlite:///yelp.db", echo=False)
query = f"""SELECT 
                b.business_id,
                name,
                address,
                city,
                state,
                b.stars,
                distance_to_cambridge,
                attributes,
                categories,
                hours,
                text
            FROM businesses as b
            LEFT JOIN tips 
                ON b.business_id = tips.business_id
            WHERE distance_to_cambridge <= 10
            AND categories LIKE "%Bubble Tea%"
            """
df = pd.read_sql(
    query,
    con=engine,
)

CPU times: user 2.27 s, sys: 1.18 s, total: 3.45 s
Wall time: 4.59 s


In [5]:
%%time
df_close = utils.load_close_businesses('cambridge', 10)

SELECT * FROM businesses WHERE distance_to_cambridge <= 10
CPU times: user 1.58 s, sys: 67.5 ms, total: 1.65 s
Wall time: 1.65 s


In [7]:
%%time
df_close_tips = utils.load_close_tips('cambridge', 10)


            SELECT 
                b.business_id,
                name,
                address,
                city,
                state,
                b.stars,
                distance_to_cambridge,
                attributes,
                categories,
                hours,
                text
            FROM businesses as b
            LEFT JOIN tips 
                ON b.business_id = tips.business_id
            WHERE distance_to_cambridge <= 10
            
CPU times: user 17.7 s, sys: 2.14 s, total: 19.8 s
Wall time: 20 s


In [45]:
%%time
df_close_reviews = utils.load_close_reviews('cambridge', 10)


            SELECT 
                businesses.business_id,
                name,
                address,
                city,
                state,
                stars,
                distance_to_cambridge,
                attributes,
                categories,
                hours,
                text
            FROM businesses 
            LEFT JOIN reviews 
                ON businesses.business_id = reviews.business_id
            WHERE distance_to_cambridge <= 10
            


OperationalError: (sqlite3.OperationalError) ambiguous column name: stars
[SQL: 
            SELECT 
                businesses.business_id,
                name,
                address,
                city,
                state,
                stars,
                distance_to_cambridge,
                attributes,
                categories,
                hours,
                text
            FROM businesses 
            LEFT JOIN reviews 
                ON businesses.business_id = reviews.business_id
            WHERE distance_to_cambridge <= 10
            ]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [369]:
_to_exclude = ['review_id', 'postal_code', 'date', 
               'attributes', 'address', 'business_id', 
               'categories', 'user_id', 'stars_x', 'stars_y',
              'latitude', 'longitude', 'is_open'
              'cool', 'funny']
cols = list(set(df_reviews_boba.columns) - set(_to_exclude))
df_reviews_boba[cols].sample(10)

,name,city,is_open,distance_to_cambridge,state,review_count,hours,useful,text,cool
1635935,Gong Cha,Allston,1,1.879708,MA,105,"{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",0,"I ordered a Strawberry Matcha Latte with boba,...",0
515715,Chatime,Boston,0,2.899948,MA,236,"{'Tuesday': '12:0-18:0', 'Wednesday': '12:0-18...",2,Bought a drink here and after taking one sip i...,0
1593439,Monkey King Tea,Malden,1,3.670842,MA,244,"{'Wednesday': '11:30-20:0', 'Thursday': '11:30...",17,This place is too bomb! I fell in love right w...,10
515583,Chatime,Boston,0,2.899948,MA,236,"{'Tuesday': '12:0-18:0', 'Wednesday': '12:0-18...",0,ChaTime is my staple boba tea place to go when...,0
948907,Dolphin Bay,Boston,1,1.756488,MA,486,"{'Monday': '0:0-0:0', 'Tuesday': '11:0-21:0', ...",0,This is probably my favorite Taiwanese restaur...,0
994068,Tealosophy,Boston,0,1.974531,MA,95,"{'Monday': '12:0-22:0', 'Tuesday': '12:0-22:0'...",0,Good place to have snow shaved ice! They are r...,0
788366,Reign Drink Lab,Dorchester,1,5.456567,MA,104,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-20:0', '...",1,We LOVE Reign. Everything is consistently abso...,1
515647,Chatime,Boston,0,2.899948,MA,236,"{'Tuesday': '12:0-18:0', 'Wednesday': '12:0-18...",5,"Legit thick, creamy real taro milk tea! None o...",1
980955,Happy Lemon Boston,Boston,1,2.800257,MA,157,"{'Monday': '11:0-19:0', 'Tuesday': '11:0-19:0'...",0,"So nice taste, and it's my favorite milk tea i...",0
762928,Gong Cha,Malden,1,4.090146,MA,27,"{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",1,"Saw this place was new and open as I drove by,...",0


In [372]:
df_boba_tips.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours', 'distance_to_cambridge', 'user_id',
       'text', 'date', 'compliment_count'],
      dtype='object')

In [376]:
_to_exclude = ['postal_code', 'date', 'hours',
               'attributes', 'address', 'business_id', 
               'categories', 'user_id', 'stars_x', 'stars_y',
              'latitude', 'longitude', 'is_open'
               'funny']
cols = list(set(df_boba_tips.columns) - set(_to_exclude))
df_boba_tips[cols].sample(10)

,name,stars,city,is_open,distance_to_cambridge,state,review_count,compliment_count,text
153,Dolphin Bay,4.5,Boston,1,1.756488,MA,486,0,Yummy food for reasonable price!!!
244,Moon Flower House,4.5,Boston,1,1.793114,MA,121,0,很好吃！环境也很干净舒服，终于找到奥尔良烤翅太开心了吧。还是那个熟悉的配方，照烧鸡腿堡，新奥...
205,Happy Lemon Boston,4.0,Boston,1,2.800257,MA,157,0,"The check in is 10% off, not $2 off"
28,Chatime,3.0,Boston,0,2.899948,MA,236,0,It is officially opened! There is a buy one ge...
96,The Juice Bar,4.0,Boston,0,2.910175,MA,159,0,Love the jasmine tea!!
50,Chatime,3.0,Boston,0,2.899948,MA,236,0,Accepts credit card--no minimum.
172,Dolphin Bay,4.5,Boston,1,1.756488,MA,486,0,Authentic Taiwanese food! Reasonable price and...
146,Dolphin Bay,4.5,Boston,1,1.756488,MA,486,0,Kicking ass MTF awesome Taiwanese food with th...
6,Pholicious,3.5,Saugus,0,7.481981,MA,21,0,"If you don't want bubbles in your bubble tea, ..."
154,Dolphin Bay,4.5,Boston,1,1.756488,MA,486,0,Rice dishes are mediocre and over-priced given...
